# Python OOP Practice — Exercises with TODOs + Quick Tests
Work through the exercises. Each task has a minimal set of self-checks using `assert`. Avoid external libs.

Topics: Encapsulation • Properties • Inheritance/Polymorphism • Abstract Base Classes • Composition •
Class/Static Methods • Dunder Methods (Operator Overloading) • Multiple Inheritance/MRO • Dependency Injection/Mocking • Dataclasses


## 1) Encapsulation + Properties — `BankAccount`
**Goal:** Encapsulate balance, validate operations, expose read-only property.
- Private attribute for balance (`__balance`).
- Methods: `deposit(amount)`, `withdraw(amount)` (raise `ValueError` if insufficient or invalid).
- Read-only property `balance`.
- Keep a simple immutable transaction log tuple list `(kind, amount, balance)`.

In [3]:
class BankAccount:
    def __init__(self, owner: str, starting_balance: float = 0.0):
        # TODO: store owner, private balance, and initialize _log as list
        if starting_balance < 0:
            raise ValueError("Starting_balance cannot be negative")
        self.owner = owner
        self.__balance = float(starting_balance)
        self._log:[tuple[str, float, float]] = []

    @property
    def balance(self) -> float:
        # TODO: return current balance
        return self.__balance

    def _add_log(self, kind: str, amount: float) -> None:
        # TODO: append to internal log
        return self._log.append((kind, float(amount), self.__balance))

    def deposit(self, amount: float) -> None:
        # TODO: validate amount > 0, update balance, log
        if amount < 0:
            raise ValueError("Deposit cannot be negative")
        self.__balance += amount
        self._add_log("deposit", amount)

    def withdraw(self, amount: float) -> None:
        # TODO: validate amount > 0 and <= balance, update, log else ValueError
        if amount > self.__balance:
            raise ValueError("Withdraw cannot be greater than balance")
        elif amount < 0:
            raise ValueError("Withdraw cannot be less than 0")
        self.__balance -= amount
        self._add_log("withdraw", amount)

    def statement(self) -> list[tuple[str, float, float]]:
        # return a copy of the log
        return self._log

# Quick tests
acc = BankAccount("Alice", 100)
acc.deposit(50)
try:
    acc.withdraw(1000)
    assert False, "Should raise for insufficient funds"
except ValueError:
    pass
acc.withdraw(30)
st = acc.statement()
assert st[-1][2] == 120 and acc.balance == 120
print("BankAccount OK")

BankAccount OK


## 2) Inheritance + Abstract Base Classes — `Shape`
**Goal:** Implement an abstract `Shape` with `area()`/`perimeter()` and concrete `Rectangle`/`Circle`.
- Use `abc.ABC` and `@abstractmethod`.
- Ensure float return; use `math` for circle.

In [2]:
from abc import ABC, abstractmethod
import math

class Shape(ABC):
    @abstractmethod
    def area(self) -> float: ...
    @abstractmethod
    def perimeter(self) -> float: ...

class Rectangle(Shape):
    def __init__(self, width: float, height: float):
        self.width = width
        self.height = height

    def area(self) -> float:
        return self.width * self.height
    def perimeter(self) -> float:
        return 2*(self.width + self.height)

class Circle(Shape):
    def __init__(self, radius: float):
        self.radius = radius

    def area(self) -> float:
        return math.pi * self.radius * self.radius

    def perimeter(self) -> float:
        return 2*math.pi*self.radius

# TODO: implement Rectangle(w,h) and Circle(r)
# Hints: rectangle area=w*h, perimeter=2*(w+h); circle area=pi*r^2, perimeter=2*pi*r

# Quick tests
r = Rectangle(3,4); c = Circle(1)
assert r.area() == 12 and r.perimeter() == 14
assert math.isclose(c.area(), math.pi, rel_tol=1e-6)
assert math.isclose(c.perimeter(), 2*math.pi, rel_tol=1e-6)
print("Fill Rectangle/Circle and un-comment tests")

Fill Rectangle/Circle and un-comment tests


## 3) Polymorphism — `speak()`
Create base `Animal.speak()` (raise `NotImplementedError`). Implement `Dog` and `Cat`.
Write a `chorus(animals)` function that returns a list of their sounds in order.

In [5]:
class Animal:
    def speak(self) -> str:
        # TODO: override in subclasses
        raise NotImplementedError

class Dog(Animal):
    # TODO: implement speak -> "Woof!"
    def speak(self) -> str:
        return "Woof!"

class Cat(Animal):
    def speak(self) -> str:
        return "Meow!"

def chorus(animals: list[Animal]) -> list[str]:
    # TODO: return list of sounds calling speak()
    return [animal.speak() for animal in animals]

# Quick tests
pets = [Dog(), Cat(), Dog()]
assert chorus(pets) == ["Woof!","Meow!","Woof!"]
print("Polymorphism ready")

Polymorphism ready


## 4) Composition — `ShoppingCart`
Design `Product(name, price)` and `CartItem(product, qty)` and `ShoppingCart(tax_rate=0.2)`.
- Methods: `add_item(product, qty)`, `remove_item(name)`, `subtotal()`, `tax()`, `total()`.
- Prefer composition (cart HAS items).

In [12]:
class Product:
    def __init__(self, name: str, price: float):
        self.name = name
        self.price = float(price)

class CartItem:
    def __init__(self, product: Product, quantity: int):
        if quantity <= 0:
            raise ValueError("qty must be > 0")
        self.product = product
        self.quantity = int(quantity)
    @property
    def subtotal(self) -> float:
        return self.product.price * self.quantity

class ShoppingCart:
    def __init__(self, tax_rate: float = 0.2):
        self.tax_rate = float(tax_rate)
        self._items: dict[str, CartItem] = {}
    def add_item(self, product: Product, qty: int) -> None:
        if product.name in self._items:
            self._items[product.name].quantity += qty
        else:
            self._items[product.name] = CartItem(product, qty)
    def remove_item(self, name: str) -> None:
        self._items.pop(name, None)
    def subtotal(self) -> float:
        return sum(item.subtotal for item in self._items.values())
    def tax(self) -> float:
        return self.subtotal() * self.tax_rate
    def total(self) -> float:
        return self.subtotal() + self.tax()

p1 = Product("Book", 10.0); p2 = Product("Pen", 1.5)
cart = ShoppingCart(tax_rate=0.19)
cart.add_item(p1, 2); cart.add_item(p2, 3)
st = cart.subtotal(); tx = cart.tax(); gt = cart.total()
assert st == 10*2 + 1.5*3 and round(tx,2) == round(st*0.19,2) and round(gt,2) == round(st+tx,2)
print("ShoppingCart: OK")

ShoppingCart: OK


## 5) Class & Static Methods — `Employee`
- Class attr `company='TechCorp'`.
- `@classmethod change_company(new)` updates the class attribute.
- `@staticmethod is_high_salary(salary)` returns bool.
- Ensure instances reflect changed class attribute.

In [ ]:
class Employee:
    company = "TechCorp"
    def __init__(self, name: str, salary: int):
        # TODO
        pass

    @classmethod
    def change_company(cls, new: str) -> None:
        # TODO
        pass

    @staticmethod
    def is_high_salary(salary: int) -> bool:
        # TODO (e.g., > 100k)
        pass

# Quick tests
# e = Employee("John", 90000)
# assert Employee.company == "TechCorp"
# Employee.change_company("NewTech")
# assert e.company == "NewTech" and not Employee.is_high_salary(e.salary)
print("Fill Employee and un-comment tests")

## 6) Dunder Methods — `Vector` with `__add__`, `__repr__`, `__eq__`
- Implement 2D vector add; equality by coordinates; readable repr.

In [15]:
class Vector:
    def __init__(self, x: float, y: float):
        # TODO
        self.x = float(x)
        self.y = float(y)

    def __add__(self, other: 'Vector') -> 'Vector':
        if not isinstance(other, Vector):
            return NotImplemented
        return Vector(self.x + other.x, self.y + other.y)

    def __eq__(self, other: object) -> bool:
        return isinstance(other, Vector) and self.x == other.x and self.y == other.y

    def __repr__(self) -> str:
        # TODO
        return f"Vector({int(self.x) if self.x.is_integer() else self.x}, {int(self.y) if self.y.is_integer() else self.y})"

# Quick tests
v1 = Vector(2,3); v2 = Vector(4,5)
assert (v1+v2) == Vector(6,8)
assert repr(v1) == "Vector(2, 3)"
print("Vector ready")

Vector ready


## 7) Multiple Inheritance & MRO — `Tracer`
Create classes A.greet() and B.greet(); C(A,B) with cooperative `super()` and show MRO effect.
Implement `Mixin` that logs calls to a list `trace` on instance.

In [21]:
class A:
    def greet(self) -> str:
        return "Hello from A"

class B:
    def greet(self) -> str:
        return "Hello from B"

class Mixin:
    def __init__(self):
        self.trace = []
        super().__init__()

class C(Mixin, A, B):
    def __init__(self):
        super().__init__()

    def greet(self) -> str:
        self.trace.append("C.greet")
        # TODO: append to trace which class ran and return first parent greet using super()
        return super().greet()

# Quick tests
c = C()
msg = c.greet()
assert msg in {"Hello from A","Hello from B"}
assert "C.greet" in c.trace
print(C.mro())
print("Implement C with cooperative super() and un-comment tests")

[<class '__main__.C'>, <class '__main__.Mixin'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>]
Implement C with cooperative super() and un-comment tests


## 8) Dependency Injection + Mocking — `ApiService`
Define a protocol-like interface for `HttpClient` with `get(url)->dict`.
Implement `ApiService(client)` that calls `client.get` and returns parsed value.
Write a tiny mock object capturing calls to assert behavior.

In [ ]:
class ApiService:
    def __init__(self, client):
        # TODO: store injected client
        pass
    def get_user_name(self, user_id: int) -> str:
        # TODO: call client.get(f"/users/{user_id}") and return payload['name']
        pass

# Quick tests with a hand-made mock
class MockClient:
    def __init__(self, data):
        self.data = data
        self.calls = []
    def get(self, url: str) -> dict:
        self.calls.append(url)
        return self.data[url]

# mock setup
# mock = MockClient({"/users/1": {"id":1,"name":"Alice"}})
# svc = ApiService(mock)
# assert svc.get_user_name(1) == "Alice"
# assert mock.calls == ["/users/1"]
print("Implement ApiService and un-comment tests")

## 9) Dataclasses — `@dataclass` for Value Objects
Refactor a small immutable value object `Point(x,y)` using `@dataclass(frozen=True)` and reuse in calculations.

In [ ]:
from dataclasses import dataclass

# TODO: implement @dataclass(frozen=True) Point with x:int, y:int and a method manhattan(other)->int

# Quick tests
# p1 = Point(1,2); p2 = Point(4,6)
# assert p1.manhattan(p2) == 7
# try:
#     p1.x = 99
#     assert False, "Should be frozen"
# except Exception:
#     pass
print("Create Point and un-comment tests")

## 10) Small Design Exercise — Extendable Logger (Strategy)
Design `Logger(level_filter, handlers)` where handlers implement `.emit(level, msg)`.
Provide `ConsoleHandler` (prints) and `MemoryHandler` (stores messages in a list). No `if/else` per handler inside Logger.

In [ ]:
class Handler:
    def emit(self, level: str, msg: str) -> None:
        raise NotImplementedError

class ConsoleHandler(Handler):
    # TODO: print message
    pass

class MemoryHandler(Handler):
    def __init__(self):
        self.store = []
    def emit(self, level: str, msg: str) -> None:
        # TODO: append to store
        pass

class Logger:
    def __init__(self, level_filter: set[str], handlers: list[Handler]):
        # TODO: assign
        pass
    def log(self, level: str, msg: str) -> None:
        # TODO: if level allowed, forward to all handlers via their emit()
        pass

# Quick tests
# mem = MemoryHandler()
# logger = Logger(level_filter={"INFO","ERROR"}, handlers=[mem])
# logger.log("DEBUG","x")      # filtered
# logger.log("INFO","hello")   # stored
# assert mem.store == [("INFO","hello")]
print("Implement Logger and un-comment tests")